# Preparação de Dados - Imóveis de São Paulo

## Propósito

Este script Python realiza a preparação e processamento de dados imobiliários da cidade de São Paulo, especificamente dados de IPTU (Imposto Predial e Territorial Urbano) e ITBI (Imposto sobre Transmissão de Bens Imóveis) para o período de 2019 a 2024. O objetivo principal é calcular o **Valor Venal** dos imóveis utilizando fórmulas complexas que consideram diversos fatores como área construída, área do terreno, profundidade, tipo de terreno, localização (Zona Fiscal), entre outros.

## Entradas (Inputs)

### Dados de IPTU
- **Arquivos Excel**: Múltiplos arquivos Excel contendo dados de IPTU com as seguintes colunas:
  - Número do Contribuinte
  - Ano do Exercício
  - Nome de Logradouro do Imóvel
  - Número do Imóvel
  - Área do Terreno
  - Área Construída
  - Área Ocupada
  - Valor do m² do Terreno
  - Valor do m² de Construção
  - Tipo de Uso do Imóvel
  - Tipo de Padrão da Construção
  - Testada para Cálculo
  - Fração Ideal
  - Fator de Obsolescência
  - Tipo de Terreno
  - Quantidade de Esquinas Frentes

### Dados de ITBI
- **Arquivos Excel**: Arquivos nomeados como `ITBI_2019.xlsx` até `ITBI_2024.xlsx`, cada um contendo planilhas mensais (JAN-YYYY, FEV-YYYY, etc.) com dados de transações imobiliárias.

### Dicionários de Mapeamento
- **fator_prof_map**: Mapeamento de profundidade para fatores de correção
- **fator_terreno**: Fatores para diferentes tipos de terreno (lote de fundos, terreno interno)
- **fator_ZF**: Fatores por Zona Fiscal (1-4)
- **fator_ea**: Fatores de excesso de área por Zona Fiscal
- **area_map**: Mapeamento de áreas por quantidade de esquinas
- **zf_map**: Mapeamento de códigos de contribuinte para Zonas Fiscais

## Saídas (Outputs)

### Arquivo CSV - IPTU
- **Nome**: `data/IPTU_2019_2024.csv`
- **Conteúdo**: Dataset processado contendo:
  - Todas as colunas originais selecionadas
  - Coluna **VALOR VENAL** calculada através da função `calcula_vv()`, que considera:
    - Valor da construção (área construída × valor m² × fator de obsolescência)
    - Valor do terreno incorporado (com ajustes por profundidade, tipo de terreno, esquina)
    - Valor do excesso de área do terreno
    - Fatores de condomínio para imóveis verticais
    - Fração ideal do imóvel

### Arquivo CSV - ITBI
- **Nome**: `ITBI_2019_2024.csv`
- **Conteúdo**: Dataset consolidado de transações imobiliárias contendo:
  - Número do Cadastro (SQL)
  - Natureza de Transação
  - Valor de Transação (declarado pelo contribuinte)
  - Data de Transação
  - Valor Venal de Referência
  - Proporção Transmitida (%)
  - Valor Venal de Referência (proporcional)
  - Base de Cálculo adotada
  - Tipo de Financiamento
  - Valor Financiado
  - Situação do SQL
  - Ano, Mês e Ano/Mês da Transação

## Processamento Realizado

1. **Carregamento e Consolidação**: Combina múltiplos arquivos de IPTU e ITBI em dataframes únicos
2. **Limpeza de Dados**: Remove registros inválidos, normaliza tipos de dados e formata campos
3. **Cálculo de Zona Fiscal**: Extrai a Zona Fiscal (ZF) a partir dos primeiros 3 dígitos do número do contribuinte
4. **Cálculo do Valor Venal**: Aplica fórmula complexa que considera múltiplos fatores de correção
5. **Análise Exploratória**: Gera estatísticas descritivas e análises de dados categóricos
6. **Exportação**: Salva os datasets processados em formato CSV para uso em análises subsequentes


In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#%cd drive/MyDrive/Projeto\ disciplina\ PCS5787
#!pwd

In [3]:
import pandas as pd
#import chardet
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
years = range(2019, 2025)

dfs = []
selected_columns = ["NUMERO DO CONTRIBUINTE",
                    "ANO DO EXERCICIO",
                    "NOME DE LOGRADOURO DO IMOVEL",
                    "NUMERO DO IMOVEL",
                    "AREA DO TERRENO",
                    "AREA CONSTRUIDA",
                    "AREA OCUPADA",
                    "VALOR DO M2 DO TERRENO",
                    "VALOR DO M2 DE CONSTRUCAO",
                    "TIPO DE USO DO IMOVEL",
                    "TIPO DE PADRAO DA CONSTRUCAO",
                    "BAIRRO DO IMOVEL",
                    "CEP DO IMOVEL",
                    "TESTADA PARA CALCULO",
                    "FRACAO IDEAL",
                    "FATOR DE OBSOLESCENCIA",
                    "TIPO DE TERRENO",
                    "QUANTIDADE DE ESQUINAS FRENTES"
                    ]

for year in years:
    file_path = f"data/IPTU_{year}.csv"
    try:
        if year < 2022:
            df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig', low_memory=False)
        else:
            df = pd.read_csv(file_path, sep=';', encoding='latin-1', low_memory=False)
        if "ï»¿NUMERO DO CONTRIBUINTE" in df.columns:
            df = df.rename(columns={"ï»¿NUMERO DO CONTRIBUINTE": "NUMERO DO CONTRIBUINTE"})
        if "QUANTIDADE DE ESQUINAS/FRENTES" in df.columns:
            df = df.rename(columns={"QUANTIDADE DE ESQUINAS/FRENTES": "QUANTIDADE DE ESQUINAS FRENTES"})

        df = df[selected_columns]
        if df['VALOR DO M2 DO TERRENO'].dtype == "object":
            df['VALOR DO M2 DO TERRENO'] = df['VALOR DO M2 DO TERRENO'].str.replace(',', '.')
        if df['VALOR DO M2 DE CONSTRUCAO'].dtype == "object":
            df['VALOR DO M2 DE CONSTRUCAO'] = df['VALOR DO M2 DE CONSTRUCAO'].str.replace(',', '.')
        if df['FATOR DE OBSOLESCENCIA'].dtype == "object":
            df['FATOR DE OBSOLESCENCIA'] = df['FATOR DE OBSOLESCENCIA'].str.replace(',', '.')
        if df['TESTADA PARA CALCULO'].dtype == "object":
            df['TESTADA PARA CALCULO'] = df['TESTADA PARA CALCULO'].str.replace(',', '.')
        if df['FRACAO IDEAL'].dtype == "object":
            df['FRACAO IDEAL'] = df['FRACAO IDEAL'].str.replace(',', '.')

        dfs.append(df)
        print(f"Successfully read {file_path}")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error reading {file_path}: {e}")



Successfully read data/IPTU_2019.csv
Successfully read data/IPTU_2020.csv
Successfully read data/IPTU_2021.csv
Successfully read data/IPTU_2022.csv
Successfully read data/IPTU_2023.csv
Successfully read data/IPTU_2024.csv


In [5]:
if dfs:
    df_combined = pd.concat(dfs, ignore_index=True)
    print("\nCombined DataFrame:")
    display(df_combined.head(3))
    display(df_combined.info())
else:
    print("No dataframes were loaded.")

del dfs


Combined DataFrame:


,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,AREA DO TERRENO,AREA CONSTRUIDA,AREA OCUPADA,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,BAIRRO DO IMOVEL,CEP DO IMOVEL,TESTADA PARA CALCULO,FRACAO IDEAL,FATOR DE OBSOLESCENCIA,TIPO DE TERRENO,QUANTIDADE DE ESQUINAS FRENTES
0,0010030001-4,2019,R S CAETANO,13.00,136,135.00,108.00,2241.00,1668.00,Loja,Comercial horizontal - padrão B,SANTA EFIGENIA,01104-001,13.00,1.0000,0.20,De esquina,1.00
1,0010030002-2,2019,R S CAETANO,19.00,90,67.00,67.00,2241.00,1668.00,Loja,Comercial horizontal - padrão B,SANTA EFIGENIA,01104-001,6.00,1.0000,0.20,Normal,0.00
2,0010030003-0,2019,R S CAETANO,27.00,105,140.00,84.00,2241.00,1668.00,Loja,Comercial horizontal - padrão B,SANTA EFIGENIA,01104-001,7.85,1.0000,0.32,Normal,0.00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21317317 entries, 0 to 21317316
Data columns (total 18 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   NUMERO DO CONTRIBUINTE          object 
 1   ANO DO EXERCICIO                int64  
 2   NOME DE LOGRADOURO DO IMOVEL    object 
 3   NUMERO DO IMOVEL                float64
 4   AREA DO TERRENO                 int64  
 5   AREA CONSTRUIDA                 float64
 6   AREA OCUPADA                    float64
 7   VALOR DO M2 DO TERRENO          object 
 8   VALOR DO M2 DE CONSTRUCAO       object 
 9   TIPO DE USO DO IMOVEL           object 
 10  TIPO DE PADRAO DA CONSTRUCAO    object 
 11  BAIRRO DO IMOVEL                object 
 12  CEP DO IMOVEL                   object 
 13  TESTADA PARA CALCULO            object 
 14  FRACAO IDEAL                    object 
 15  FATOR DE OBSOLESCENCIA          object 
 16  TIPO DE TERRENO                 object 
 17  QUANTIDADE DE ESQUINAS FR

None

In [ ]:
data_types = {
    "NUMERO DO CONTRIBUINTE": "string",
    "ANO DO EXERCICIO": "string",
    "NOME DE LOGRADOURO DO IMOVEL": "string",
    "NUMERO DO IMOVEL": "Int64",
    "AREA DO TERRENO": "float64",
    "VALOR DO M2 DO TERRENO": "Int64",
    "VALOR DO M2 DE CONSTRUCAO": "Int64",
    "TIPO DE USO DO IMOVEL": "string",
    "TIPO DE PADRAO DA CONSTRUCAO": "string",
    "BAIRRO DO IMOVEL": "string",
    "CEP DO IMOVEL": "string",
    "TESTADA PARA CALCULO": "float64",
    "FRACAO IDEAL": "float64",
    "FATOR DE OBSOLESCENCIA": "float64",
    "QUANTIDADE DE ESQUINAS FRENTES": "Int64",
    "TIPO DE TERRENO": "string",
}

for col, dtype in data_types.items():
    # Trata erros potenciais durante a conversão
    try:
        if dtype == 'string':
            df_combined[col] = df_combined[col].astype(str)
        elif dtype == 'Int64':
            df_combined[col] = pd.to_numeric(df_combined[col], errors='coerce').astype('Int64')
        else:
            df_combined[col] = df_combined[col].astype(dtype)
        print(f"Successfully converted {col} to {dtype}")
    except Exception as e:
        print(f"Error converting {col} to {dtype}: {e}")

df_combined['NUMERO DO CONTRIBUINTE'] = df_combined['NUMERO DO CONTRIBUINTE'].str.replace(r'[^A-Za-z0-9 ]', '', regex=True)
display(df_combined.info())
display(df_combined.head())

Successfully converted NUMERO DO CONTRIBUINTE to string
Successfully converted ANO DO EXERCICIO to string
Successfully converted NOME DE LOGRADOURO DO IMOVEL to string
Successfully converted NUMERO DO IMOVEL to Int64
Successfully converted AREA DO TERRENO to float64
Successfully converted VALOR DO M2 DO TERRENO to Int64
Successfully converted VALOR DO M2 DE CONSTRUCAO to Int64
Successfully converted TIPO DE USO DO IMOVEL to string
Successfully converted TIPO DE PADRAO DA CONSTRUCAO to string
Successfully converted BAIRRO DO IMOVEL to string
Successfully converted CEP DO IMOVEL to string
Error converting TESTADA PARA CALCULO to float64: could not convert string to float: 'Residência'
Successfully converted FRACAO IDEAL to float64
Successfully converted FATOR DE OBSOLESCENCIA to float64
Error converting QUANTIDADE DE ESQUINAS FRENTES to Int64: cannot safely cast non-equivalent float64 to int64
Successfully converted TIPO DE TERRENO to string
<class 'pandas.core.frame.DataFrame'>
RangeInd

None

,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,AREA DO TERRENO,AREA CONSTRUIDA,AREA OCUPADA,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,BAIRRO DO IMOVEL,CEP DO IMOVEL,TESTADA PARA CALCULO,FRACAO IDEAL,FATOR DE OBSOLESCENCIA,TIPO DE TERRENO,QUANTIDADE DE ESQUINAS FRENTES
0,00100300014,2019,R S CAETANO,13,136.00,135.00,108.00,2241,1668,Loja,Comercial horizontal - padrão B,SANTA EFIGENIA,01104-001,13.00,1.00,0.20,De esquina,1.00
1,00100300022,2019,R S CAETANO,19,90.00,67.00,67.00,2241,1668,Loja,Comercial horizontal - padrão B,SANTA EFIGENIA,01104-001,6.00,1.00,0.20,Normal,0.00
2,00100300030,2019,R S CAETANO,27,105.00,140.00,84.00,2241,1668,Loja,Comercial horizontal - padrão B,SANTA EFIGENIA,01104-001,7.85,1.00,0.32,Normal,0.00
3,00100300049,2019,R S CAETANO,33,108.00,103.00,86.00,2241,1668,Loja,Comercial horizontal - padrão B,nan,01104-001,6.05,1.00,0.20,Normal,0.00
4,00100300057,2019,R S CAETANO,39,120.00,98.00,96.00,2241,1668,Loja,Comercial horizontal - padrão B,nan,01104-001,6.70,1.00,0.20,Normal,0.00


In [7]:
df_combined.head(5)

,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,AREA DO TERRENO,AREA CONSTRUIDA,AREA OCUPADA,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,BAIRRO DO IMOVEL,CEP DO IMOVEL,TESTADA PARA CALCULO,FRACAO IDEAL,FATOR DE OBSOLESCENCIA,TIPO DE TERRENO,QUANTIDADE DE ESQUINAS FRENTES
0,00100300014,2019,R S CAETANO,13,136.00,135.00,108.00,2241,1668,Loja,Comercial horizontal - padrão B,SANTA EFIGENIA,01104-001,13.00,1.00,0.20,De esquina,1.00
1,00100300022,2019,R S CAETANO,19,90.00,67.00,67.00,2241,1668,Loja,Comercial horizontal - padrão B,SANTA EFIGENIA,01104-001,6.00,1.00,0.20,Normal,0.00
2,00100300030,2019,R S CAETANO,27,105.00,140.00,84.00,2241,1668,Loja,Comercial horizontal - padrão B,SANTA EFIGENIA,01104-001,7.85,1.00,0.32,Normal,0.00
3,00100300049,2019,R S CAETANO,33,108.00,103.00,86.00,2241,1668,Loja,Comercial horizontal - padrão B,nan,01104-001,6.05,1.00,0.20,Normal,0.00
4,00100300057,2019,R S CAETANO,39,120.00,98.00,96.00,2241,1668,Loja,Comercial horizontal - padrão B,nan,01104-001,6.70,1.00,0.20,Normal,0.00


### Preenche bairros faltantes com a moda do bairro no setor 


In [8]:
#cria os atributos de localicação baseado no numero do contribuinte
df_combined['NUMERO DO CONTRIBUINTE'] = df_combined['NUMERO DO CONTRIBUINTE'].astype(str).str.replace('-', '')
df_combined['NUMERO DO CONTRIBUINTE'] = df_combined['NUMERO DO CONTRIBUINTE'].astype(str).str.zfill(11)
df_combined = df_combined[pd.to_numeric(df_combined['TESTADA PARA CALCULO'], errors='coerce').notna()]
df_combined['setor'] = df_combined['NUMERO DO CONTRIBUINTE'].astype(str).str[:3]
df_combined['quadra'] = df_combined['NUMERO DO CONTRIBUINTE'].astype(str).str[3:6]
df_combined['lote'] = df_combined['NUMERO DO CONTRIBUINTE'].astype(str).str[6:10]

In [9]:
base_bairro = (
    df_combined
    .groupby('setor', as_index = False)
    .agg(bairro = ('BAIRRO DO IMOVEL', lambda x: x.mode().iloc[0] if not x.mode().empty else None))
)

In [10]:
df_combined = (
    df_combined
    .merge(
        base_bairro,
        on = 'setor',
        how = 'left'
        )
)

In [11]:
df_combined.head(3)

,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,AREA DO TERRENO,AREA CONSTRUIDA,AREA OCUPADA,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,TIPO DE USO DO IMOVEL,...,CEP DO IMOVEL,TESTADA PARA CALCULO,FRACAO IDEAL,FATOR DE OBSOLESCENCIA,TIPO DE TERRENO,QUANTIDADE DE ESQUINAS FRENTES,setor,quadra,lote,bairro
0,00100300014,2019,R S CAETANO,13,136.00,135.00,108.00,2241,1668,Loja,...,01104-001,13.00,1.00,0.20,De esquina,1.00,001,003,0001,nan
1,00100300022,2019,R S CAETANO,19,90.00,67.00,67.00,2241,1668,Loja,...,01104-001,6.00,1.00,0.20,Normal,0.00,001,003,0002,nan
2,00100300030,2019,R S CAETANO,27,105.00,140.00,84.00,2241,1668,Loja,...,01104-001,7.85,1.00,0.32,Normal,0.00,001,003,0003,nan


In [12]:
display(df_combined.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21317311 entries, 0 to 21317310
Data columns (total 22 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   NUMERO DO CONTRIBUINTE          object 
 1   ANO DO EXERCICIO                object 
 2   NOME DE LOGRADOURO DO IMOVEL    object 
 3   NUMERO DO IMOVEL                Int64  
 4   AREA DO TERRENO                 float64
 5   AREA CONSTRUIDA                 float64
 6   AREA OCUPADA                    float64
 7   VALOR DO M2 DO TERRENO          Int64  
 8   VALOR DO M2 DE CONSTRUCAO       Int64  
 9   TIPO DE USO DO IMOVEL           object 
 10  TIPO DE PADRAO DA CONSTRUCAO    object 
 11  BAIRRO DO IMOVEL                object 
 12  CEP DO IMOVEL                   object 
 13  TESTADA PARA CALCULO            object 
 14  FRACAO IDEAL                    float64
 15  FATOR DE OBSOLESCENCIA          float64
 16  TIPO DE TERRENO                 object 
 17  QUANTIDADE DE ESQUINAS FR

None

In [13]:
fator_prof_map = {
    0: 0.7071, 1: 0.7071, 2: 0.7071, 3: 0.7071, 4: 0.7071,
    5: 0.7071, 6: 0.7071, 7: 0.7071, 8: 0.7071, 9: 0.7071,
    10: 0.7071, 11: 0.7416, 12: 0.7746, 13: 0.8062, 14: 0.8367,
    15: 0.8660, 16: 0.8944, 17: 0.9220, 18: 0.9487, 19: 0.9747,
    20: 1.0000, 21: 1.0000, 22: 1.0000, 23: 1.0000, 24: 1.0000,
    25: 1.0000, 26: 1.0000, 27: 1.0000, 28: 1.0000, 29: 1.0000,
    30: 1.0000, 31: 1.0000, 32: 1.0000, 33: 1.0000, 34: 1.0000,
    35: 1.0000, 36: 1.0000, 37: 1.0000, 38: 1.0000, 39: 1.0000,
    40: 1.0000, 41: 0.9877, 42: 0.9759, 43: 0.9645, 44: 0.9535,
    45: 0.9428, 46: 0.9325, 47: 0.9225, 48: 0.9129, 49: 0.9035,
    50: 0.8944, 51: 0.8856, 52: 0.8771, 53: 0.8687, 54: 0.8607,
    55: 0.8528, 56: 0.8452, 57: 0.8377, 58: 0.8305, 59: 0.8234,
    60: 0.8165, 61: 0.8098, 62: 0.8032, 63: 0.7968, 64: 0.7906,
    65: 0.7845, 66: 0.7785, 67: 0.7727, 68: 0.7670, 69: 0.7614,
    70: 0.7559, 71: 0.7506, 72: 0.7454, 73: 0.7402, 74: 0.7352,
    75: 0.7303, 76: 0.7255, 77: 0.7207, 78: 0.7161, 79: 0.7116,
    80: 0.7071, 81: 0.6984, 82: 0.6984, 83: 0.6901, 84: 0.6901,
    85: 0.6820, 86: 0.6820, 87: 0.6742, 88: 0.6742, 89: 0.6667,
    90: 0.6667, 91: 0.6594, 92: 0.6594, 93: 0.6523, 94: 0.6523,
    95: 0.6455, 96: 0.6455, 97: 0.6389, 98: 0.6389, 99: 0.6325,
    100: 0.6325, 101: 0.6172, 102: 0.6172, 103: 0.6172, 104: 0.6172,
    105: 0.6172, 106: 0.6030, 107: 0.6030, 108: 0.6030, 109: 0.6030,
    110: 0.6030, 111: 0.5898, 112: 0.5898, 113: 0.5898, 114: 0.5898,
    115: 0.5898, 116: 0.5774, 117: 0.5774, 118: 0.5774, 119: 0.5774,
    120: 0.5774, 121: 0.5657, 122: 0.5657, 123: 0.5657, 124: 0.5657,
    125: 0.5657, 126: 0.5547, 127: 0.5547, 128: 0.5547, 129: 0.5547,
    130: 0.5547, 131: 0.5443, 132: 0.5443, 133: 0.5443, 134: 0.5443,
    135: 0.5443, 136: 0.5345, 137: 0.5345, 138: 0.5345, 139: 0.5345,
    140: 0.5345, 141: 0.5252, 142: 0.5252, 143: 0.5252, 144: 0.5252,
    145: 0.5252, 146: 0.5164, 147: 0.5164, 148: 0.5164, 149: 0.5164,
    150: 0.5164, 151: 0.5000, 152: 0.5000, 153: 0.5000, 154: 0.5000,
    155: 0.5000, 156: 0.5000, 157: 0.5000, 158: 0.5000, 159: 0.5000,
    160: 0.5000, 161: 0.4851, 162: 0.4851, 163: 0.4851, 164: 0.4851,
    165: 0.4851, 166: 0.4851, 167: 0.4851, 168: 0.4851, 169: 0.4851,
    170: 0.4851, 171: 0.4714, 172: 0.4714, 173: 0.4714, 174: 0.4714,
    175: 0.4714, 176: 0.4714, 177: 0.4714, 178: 0.4714, 179: 0.4714,
    180: 0.4714, 181: 0.4588, 182: 0.4588, 183: 0.4588, 184: 0.4588,
    185: 0.4588, 186: 0.4588, 187: 0.4588, 188: 0.4588, 189: 0.4588,
    190: 0.4588, 191: 0.4472, 192: 0.4472, 193: 0.4472, 194: 0.4472,
    195: 0.4472, 196: 0.4472, 197: 0.4472, 198: 0.4472, 199: 0.4472,
    200: 0.4472
}

fator_terreno = {
    'Lote de fundos': 0.6,
    'Terreno interno': 0.7,
}

fator_ZF = {
    1: 1.3,
    2: 1.2,
    3: 1.1,
    4: 1.1
}

fator_ea = {
    1: 3,
    2: 5,
    3: 10,
    4: 10
}

area_map = {
    1: 900,
    2: 1800,
    3: 2700,
    4: 3600
}

zf_map = {
    "001": 1, "002": 1, "003": 1, "004": 2, "005": 1, "006": 1, "007": 1, "008": 1,
    "009": 1, "010": 1, "011": 1, "012": 1, "013": 1, "014": 1, "015": 1, "016": 1,
    "017": 2, "018": 2, "019": 2, "020": 1, "021": 1, "022": 1, "023": 1, "024": 1,
    "025": 1, "026": 2, "027": 2, "028": 2, "029": 2, "030": 1, "031": 2, "032": 2,
    "033": 1, "034": 1, "035": 1, "036": 1, "037": 1, "038": 1, "039": 1, "040": 1,
    "041": 1, "042": 1, "043": 2, "044": 2, "045": 1, "046": 1, "047": 1, "048": 2,
    "049": 2, "050": 2, "051": 2, "052": 2, "053": 2, "054": 1, "055": 2, "056": 2,
    "057": 2, "058": 2, "059": 2, "060": 2, "061": 2, "062": 2, "063": 3, "064": 2,
    "065": 2, "066": 2, "067": 2, "068": 2, "069": 1, "070": 2, "071": 2, "072": 1,
    "073": 1, "074": 2, "075": 2, "076": 2, "077": 2, "078": 2, "079": 2, "080": 1,
    "081": 1, "082": 2, "083": 1, "084": 1, "085": 1, "086": 1, "087": 2, "088": 2,
    "089": 2, "090": 2, "091": 2, "093": 2, "094": 3, "095": 2, "096": 1, "097": 2,
    "098": 2, "099": 2, "100": 2, "101": 2, "102": 2, "103": 2, "104": 2, "105": 2,
    "106": 3, "107": 2, "108": 2, "109": 2, "110": 2, "111": 3, "112": 3, "113": 2,
    "114": 3, "115": 3, "116": 2, "117": 2, "118": 2, "119": 2, "120": 2, "121": 2,
    "122": 2, "123": 2, "124": 3, "125": 2, "126": 3, "127": 3, "128": 3, "129": 3,
    "130": 3, "131": 3, "132": 3, "133": 3, "134": 3, "135": 3, "136": 3, "137": 3,
    "138": 3, "139": 3, "140": 3, "141": 3, "142": 3, "143": 3, "144": 3, "145": 3,
    "146": 3, "147": 3, "148": 2, "149": 3, "150": 3, "151": 3, "152": 3, "153": 3,
    "154": 3, "155": 3, "156": 3, "157": 3, "158": 3, "159": 2, "160": 2, "161": 3,
    "162": 3, "163": 2, "164": 3, "165": 3, "166": 3, "167": 3, "168": 3, "169": 3,
    "170": 2, "171": 2, "172": 3, "173": 3, "174": 3, "175": 3, "176": 3, "177": 3,
    "178": 3, "179": 3, "180": 3, "181": 3, "182": 3, "183": 3, "184": 3, "185": 3,
    "186": 3, "187": 3, "188": 3, "189": 3, "190": 3, "191": 3, "192": 3, "193": 3,
    "194": 3, "195": 3, "196": 2, "197": 2, "198": 3, "199": 2, "200": 1, "201": 4,
    "202": 4, "203": 4, "204": 4, "205": 4, "206": 4, "207": 4, "208": 4, "209": 3,
    "210": 4, "211": 4, "212": 3, "213": 4, "214": 3, "215": 4, "216": 4, "217": 4,
    "218": 4, "219": 4, "220": 4, "221": 4, "222": 4, "223": 4, "224": 4, "225": 4,
    "226": 4, "227": 4, "228": 4, "229": 3, "230": 3, "231": 4, "232": 4, "233": 3,
    "234": 3, "235": 3, "236": 4, "237": 3, "238": 4, "239": 4, "240": 3, "241": 4,
    "242": 3, "243": 4, "244": 4, "245": 4, "246": 4, "247": 4, "248": 4, "249": 4,
    "250": 4, "251": 4, "252": 3, "253": 3, "254": 4, "255": 4, "256": 4, "257": 4,
    "258": 4, "259": 4, "260": 4, "261": 4, "262": 4, "263": 4, "264": 4, "265": 4,
    "266": 4, "267": 4, "268": 4, "269": 4, "270": 4, "271": 4, "272": 4, "273": 4,
    "274": 4, "275": 4, "276": 4, "277": 4, "278": 4, "279": 4, "280": 4, "281": 4,
    "282": 4, "283": 4, "284": 4, "285": 4, "286": 4, "287": 4, "288": 4, "289": 4,
    "290": 4, "291": 4, "292": 4, "293": 4, "294": 4, "295": 4, "296": 4, "297": 4,
    "298": 4, "299": 1, "300": 1, "301": 2, "302": 2, "303": 2, "304": 2, "305": 2,
    "306": 2, "307": 3, "308": 3, "309": 1, "310": 2
}
df_combined['ZF'] = df_combined.apply(lambda row: zf_map[row['NUMERO DO CONTRIBUINTE'].zfill(11)[:3]], axis=1)

In [14]:
def calcula_vv(row):
  vv_c = float(row['AREA CONSTRUIDA'])*float(row['VALOR DO M2 DE CONSTRUCAO'])*float(row['FATOR DE OBSOLESCENCIA'])

  area_incorporada = float(row['AREA DO TERRENO'])
  excesso_area = 0
  if float(row['AREA OCUPADA']) < row['AREA DO TERRENO']/fator_ea[row['ZF']]:
    area_incorporada = (fator_ea[row['ZF']]+1)*row['AREA OCUPADA']
    excesso_area = row['AREA DO TERRENO'] - area_incorporada

  fator_prof = 1
  if float(row['TESTADA PARA CALCULO']) > 0.1:
    fator_prof = fator_prof_map[min(int(float(row['AREA DO TERRENO'])/float(row['TESTADA PARA CALCULO'])),200)]

  vv_t = area_incorporada*float(row['VALOR DO M2 DO TERRENO'])*fator_prof
  vv_e = excesso_area*float(row['VALOR DO M2 DO TERRENO'])*fator_prof

  if row['TIPO DE TERRENO'] in fator_terreno:
    vv_t = vv_t*fator_terreno[row['TIPO DE TERRENO']]
    vv_e = vv_e*fator_terreno[row['TIPO DE TERRENO']]

  fator_esq = 1
  area_esq = 0
  if row['TIPO DE TERRENO'] == 'De esquina' or row['TIPO DE TERRENO'] == 'Lote de esquina em ZER':
    area_esq = min(area_map[min(row['QUANTIDADE DE ESQUINAS FRENTES'],4)], row['AREA DO TERRENO'])
    fator_esq = 1+(fator_ZF[row['ZF']]-1)*area_esq/row['AREA DO TERRENO']

  vv_t = vv_t*fator_esq
  vv_e = vv_e*fator_esq

  fator_condo = 1

  if row['FRACAO IDEAL'] != 1 and (row['TIPO DE PADRAO DA CONSTRUCAO'].startswith('Residencial vertical') or row['TIPO DE PADRAO DA CONSTRUCAO'].startswith('Comercial vertical')) and row['TIPO DE USO DO IMOVEL'] != 'TERRENO':
      teste_condo = vv_t*float(row['FRACAO IDEAL'])*1.6/vv_c
      if teste_condo < 0.2:
          fator_condo = (2.2-3*teste_condo)
      elif teste_condo < 2.01:
          fator_condo = 1.6
      elif teste_condo < 7:
          fator_condo = 1.8-teste_condo/10
      else:
          fator_condo = 1.1
      vv_t = vv_t*fator_condo
      vv_e = vv_e*fator_condo

  vv_t = vv_t*float(row['FRACAO IDEAL'])
  vv_e = vv_e*float(row['FRACAO IDEAL'])

  # print(fator_condo)
  # print(fator_prof)
  # print(fator_esq)
  # print(vv_c)
  # print(vv_t)
  # print(vv_e)
  return vv_c+vv_t+vv_e

In [ ]:
data_types = {
    "NUMERO DO CONTRIBUINTE": "string",
    "ANO DO EXERCICIO": "string",
    "NOME DE LOGRADOURO DO IMOVEL": "string",
    "NUMERO DO IMOVEL": "Int64",
    "AREA DO TERRENO": "float64",
    "VALOR DO M2 DO TERRENO": "Int64",
    "VALOR DO M2 DE CONSTRUCAO": "Int64",
    "TIPO DE USO DO IMOVEL": "string",
    "TIPO DE PADRAO DA CONSTRUCAO": "string",
    "BAIRRO DO IMOVEL": "string",
    "CEP DO IMOVEL": "string",
    "TESTADA PARA CALCULO": "float64",
    "FRACAO IDEAL": "float64",
    "FATOR DE OBSOLESCENCIA": "float64",
    "QUANTIDADE DE ESQUINAS FRENTES": "Int64",
    "TIPO DE TERRENO": "string",
}

for col, dtype in data_types.items():
    # Trata erros potenciais durante a conversão
    try:
        if dtype == 'string':
            df_combined[col] = df_combined[col].astype(str)
        elif dtype == 'Int64':
            df_combined[col] = pd.to_numeric(df_combined[col], errors='coerce').astype('Int64')
        else:
            df_combined[col] = df_combined[col].astype(dtype)
        print(f"Successfully converted {col} to {dtype}")
    except Exception as e:
        print(f"Error converting {col} to {dtype}: {e}")

df_combined['NUMERO DO CONTRIBUINTE'] = df_combined['NUMERO DO CONTRIBUINTE'].str.replace(r'[^A-Za-z0-9 ]', '', regex=True)
display(df_combined.info())
display(df_combined.head())

Successfully converted NUMERO DO CONTRIBUINTE to string
Successfully converted ANO DO EXERCICIO to string
Successfully converted NOME DE LOGRADOURO DO IMOVEL to string
Successfully converted NUMERO DO IMOVEL to Int64
Successfully converted AREA DO TERRENO to float64
Successfully converted VALOR DO M2 DO TERRENO to Int64
Successfully converted VALOR DO M2 DE CONSTRUCAO to Int64
Successfully converted TIPO DE USO DO IMOVEL to string
Successfully converted TIPO DE PADRAO DA CONSTRUCAO to string
Successfully converted BAIRRO DO IMOVEL to string
Successfully converted CEP DO IMOVEL to string
Successfully converted TESTADA PARA CALCULO to float64
Successfully converted FRACAO IDEAL to float64
Successfully converted FATOR DE OBSOLESCENCIA to float64
Successfully converted QUANTIDADE DE ESQUINAS FRENTES to Int64
Successfully converted TIPO DE TERRENO to string
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21317311 entries, 0 to 21317310
Data columns (total 23 columns):
 #   Column         

None

,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,AREA DO TERRENO,AREA CONSTRUIDA,AREA OCUPADA,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,TIPO DE USO DO IMOVEL,...,TESTADA PARA CALCULO,FRACAO IDEAL,FATOR DE OBSOLESCENCIA,TIPO DE TERRENO,QUANTIDADE DE ESQUINAS FRENTES,setor,quadra,lote,bairro,ZF
0,00100300014,2019,R S CAETANO,13,136.00,135.00,108.00,2241,1668,Loja,...,13.00,1.00,0.20,De esquina,1,001,003,0001,nan,1
1,00100300022,2019,R S CAETANO,19,90.00,67.00,67.00,2241,1668,Loja,...,6.00,1.00,0.20,Normal,0,001,003,0002,nan,1
2,00100300030,2019,R S CAETANO,27,105.00,140.00,84.00,2241,1668,Loja,...,7.85,1.00,0.32,Normal,0,001,003,0003,nan,1
3,00100300049,2019,R S CAETANO,33,108.00,103.00,86.00,2241,1668,Loja,...,6.05,1.00,0.20,Normal,0,001,003,0004,nan,1
4,00100300057,2019,R S CAETANO,39,120.00,98.00,96.00,2241,1668,Loja,...,6.70,1.00,0.20,Normal,0,001,003,0005,nan,1


In [16]:
df_combined['VALOR VENAL'] = df_combined.apply(lambda row: calcula_vv(row), axis=1)

In [18]:
selected_columns = ["NUMERO DO CONTRIBUINTE",
                    "ANO DO EXERCICIO",
                    "NOME DE LOGRADOURO DO IMOVEL",
                    "NUMERO DO IMOVEL",
                    "AREA DO TERRENO",
                    "AREA CONSTRUIDA",
                    "AREA OCUPADA",
                    "VALOR DO M2 DO TERRENO",
                    "VALOR DO M2 DE CONSTRUCAO",
                    "TIPO DE USO DO IMOVEL",
                    "TIPO DE PADRAO DA CONSTRUCAO",
                    "BAIRRO DO IMOVEL",
                    "CEP DO IMOVEL",
                    "VALOR VENAL",
                    "setor",
                    "quadra",
                    "lote",
                    "bairro"
                    ]
df_combined = df_combined[selected_columns]

In [19]:
df_combined.head(3)

,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,AREA DO TERRENO,AREA CONSTRUIDA,AREA OCUPADA,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,BAIRRO DO IMOVEL,CEP DO IMOVEL,VALOR VENAL,setor,quadra,lote,bairro
0,00100300014,2019,R S CAETANO,13,136.00,135.00,108.00,2241,1668,Loja,Comercial horizontal - padrão B,SANTA EFIGENIA,01104-001,325195.24,001,003,0001,nan
1,00100300022,2019,R S CAETANO,19,90.00,67.00,67.00,2241,1668,Loja,Comercial horizontal - padrão B,SANTA EFIGENIA,01104-001,197014.74,001,003,0002,nan
2,00100300030,2019,R S CAETANO,27,105.00,140.00,84.00,2241,1668,Loja,Comercial horizontal - padrão B,SANTA EFIGENIA,01104-001,264429.29,001,003,0003,nan


In [20]:
df_combined.to_csv('data/IPTU_2019_2024.csv', index=False)

## Gera estatísticas descritivas


In [21]:

numerical_columns = df_combined.select_dtypes(include=['int64', 'float64'])
descriptive_stats = numerical_columns.describe()
display(descriptive_stats)

,NUMERO DO IMOVEL,AREA DO TERRENO,AREA CONSTRUIDA,AREA OCUPADA,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,VALOR VENAL
count,20804861.00,21317311.00,21317311.00,21317311.00,21317311.00,21317311.00,21317311.00
mean,581.87,3986.79,154.29,1304.12,2462.21,1846.71,437341.05
std,1124.40,16923.68,1057.46,3451.62,3144.23,819.97,4474126.24
min,0.00,1.00,0.00,0.00,45.00,0.00,78.01
25%,102.00,185.00,63.00,90.00,682.00,1322.00,112866.61
50%,262.00,711.00,100.00,324.00,1331.00,1774.00,209142.38
75%,619.00,2481.00,153.00,1234.00,2870.00,2291.00,379048.16
max,99541.00,4307493.00,950328.00,464200.00,32871.00,4249.00,4861333058.59


## Analisa dados categóricos


In [ ]:
categorical_columns = df_combined.select_dtypes(include='object').columns.tolist()

categorical_columns.remove('NUMERO DO CONTRIBUINTE')
categorical_columns.remove('ANO DO EXERCICIO')
#categorical_columns.remove('NUMERO DA NL')
#categorical_columns.remove('CODLOG DO IMOVEL')

for col in categorical_columns:
    print(f"\nValue counts for column: {col}")
    # Exibe as 20 principais categorias para colunas com muitos valores únicos
    if df_combined[col].nunique() > 20:
        display(df_combined[col].value_counts().head(20))
    else:
        display(df_combined[col].value_counts())


Value counts for column: NOME DE LOGRADOURO DO IMOVEL


NOME DE LOGRADOURO DO IMOVEL
AV  PAULISTA                         77929
AV  RAIMUNDO PEREIRA DE MAGALHAES    55815
R DA CONSOLACAO                      48862
AV BRIG LUIS ANTONIO                 46957
R  VERGUEIRO                         46871
AV  NOVE DE JULHO                    43988
AV BRIG FARIA LIMA                   43365
AV  CELSO GARCIA                     40689
R  BELA CINTRA                       37504
R  VOLUNTARIOS DA PATRIA             37337
AV  ANGELICA                         36800
AV  INTERLAGOS                       35613
R FREI CANECA                        33456
AL  SANTOS                           31434
AL  LORENA                           29945
AV DAS NACOES UNIDAS                 28548
AV PDE ARLINDO VIEIRA                28472
AV  VILA EMA                         27459
R  AUGUSTA                           26127
R  OSCAR FREIRE                      25828
Name: count, dtype: int64


Value counts for column: TIPO DE USO DO IMOVEL


TIPO DE USO DO IMOVEL
Apartamento em condomínio                                                                     8521073
Residência                                                                                    6491698
Residência coletiva, exclusive cortiço (mais de uma residência no lote)                       1225981
Garagem (unidade autônoma) em edifício em condomínio de uso exclusivamente residencial        1185071
Escritório/consultório em condomínio (unidade autônoma)                                        831480
Terreno                                                                                        628525
Residência e outro uso (predominância residencial)                                             512335
Loja                                                                                           324601
Loja e residência (predominância comercial)                                                    266636
Garagem (unidade autônoma) em edifício em condomínio de escr


Value counts for column: TIPO DE PADRAO DA CONSTRUCAO


...

quadra
006    161801
025    154186
014    153880
035    153615
023    149283
036    145814
019    144106
064    139168
048    137668
009    135699
012    135067
005    133841
003    132772
061    131521
046    130847
021    128756
016    128601
002    127780
024    127609
029    127425
Name: count, dtype: int64


Value counts for column: lote


lote
0008    170900
0007    170473
0006    170458
0005    170214
0004    169940
0010    169394
0009    169359
0003    168458
0011    167455
0012    167248
0013    166560
0014    165753
0015    165501
0002    165105
0017    164301
0016    164231
0018    162976
0019    161548
0020    161168
0021    160583
Name: count, dtype: int64


Value counts for column: bairro


bairro
nan                       19558367
PERDIZES                    170331
CJ HAB JOSE BONIFACIO       148389
ITAQUERA                    124110
AMERICANOPOLIS              106706
STA ETELVINA VII             94915
CONJ PE M DA NOBREGA         80386
STA ETELVINA IIA             79062
FAZENDA DA JUTA              76766
C H PRESID JUSCELINO         66475
COHAB ADVENTISTA             62667
CJ HAB BRIG F LIMA           61854
CAPELA DO SOCORRO            52116
JD BARRO BRANCO I            51790
JD CONQUISTA                 44666
VL IOLANDA                   40202
CJ HAB RAPOSO TAVARES        37703
ITAIM PAULISTA               29044
ALTO DE PINHEIROS            28936
CJ HAB INACIO MONTEIRO       27477
Name: count, dtype: int64

### Gera arquivo unico com dados de ITPI

In [37]:
years = range(2019, 2025)
months = ['JAN', 'FEV', 'MAR', 'ABR', 'MAI', 'JUN',
          'JUL', 'AGO', 'SET', 'OUT', 'NOV', 'DEZ']

dfs_itbi = []

selected_columns = ["N° do Cadastro (SQL)",
                    "Natureza de Transação",
                    "Valor de Transação (declarado pelo contribuinte)",
                    "Data de Transação",
                    "Valor Venal de Referência",
                    "Proporção Transmitida (%)",
                    "Valor Venal de Referência (proporcional)",
                    "Base de Cálculo adotada",
                    "Tipo de Financiamento",
                    "Valor Financiado",
                    "Situação do SQL",
                    "Área do Terreno (m2)",
                    "Área Construída (m2)"
                    ]

for year in years:
    file_path = f"data/xlsx/ITBI_{year}.xlsx"
    df = None
    for month in months:
      sheet_name = f"{month}-{year}"
      try:
          df = pd.read_excel(file_path, sheet_name=sheet_name)
          df = df[selected_columns]
          print(f"Successfully read {file_path} {sheet_name}")
      except FileNotFoundError:
          print(f"File not found: {file_path}, {sheet_name}")
      except Exception as e:
          print(f"Error reading {file_path}, {sheet_name}: {e}")
      dfs_itbi.append(df)

Successfully read data/xlsx/ITBI_2019.xlsx JAN-2019
Successfully read data/xlsx/ITBI_2019.xlsx FEV-2019
Successfully read data/xlsx/ITBI_2019.xlsx MAR-2019
Successfully read data/xlsx/ITBI_2019.xlsx ABR-2019
Successfully read data/xlsx/ITBI_2019.xlsx MAI-2019
Successfully read data/xlsx/ITBI_2019.xlsx JUN-2019
Successfully read data/xlsx/ITBI_2019.xlsx JUL-2019
Successfully read data/xlsx/ITBI_2019.xlsx AGO-2019
Successfully read data/xlsx/ITBI_2019.xlsx SET-2019
Successfully read data/xlsx/ITBI_2019.xlsx OUT-2019
Successfully read data/xlsx/ITBI_2019.xlsx NOV-2019
Successfully read data/xlsx/ITBI_2019.xlsx DEZ-2019
Successfully read data/xlsx/ITBI_2020.xlsx JAN-2020
Successfully read data/xlsx/ITBI_2020.xlsx FEV-2020
Successfully read data/xlsx/ITBI_2020.xlsx MAR-2020
Successfully read data/xlsx/ITBI_2020.xlsx ABR-2020
Successfully read data/xlsx/ITBI_2020.xlsx MAI-2020
Successfully read data/xlsx/ITBI_2020.xlsx JUN-2020
Successfully read data/xlsx/ITBI_2020.xlsx JUL-2020
Successfully

In [24]:
if dfs_itbi:
    df_combined_itbi = pd.concat(dfs_itbi, ignore_index=True)
    print("\nCombined DataFrame:")
    display(df_combined_itbi.head())
    display(df_combined_itbi.info())
else:
    print("No dataframes were loaded.")


Combined DataFrame:


,N° do Cadastro (SQL),Natureza de Transação,Valor de Transação (declarado pelo contribuinte),Data de Transação,Valor Venal de Referência,Proporção Transmitida (%),Valor Venal de Referência (proporcional),Base de Cálculo adotada,Tipo de Financiamento,Valor Financiado,...,173753,Ativo Territorial,18467,47.63,0.1,0.2,TERRENO,0.3,TERRENO.1,Unnamed: 27
0,8010602450,1.Compra e venda,600000.00,2018-12-19 00:00:00,518060.00,100.00,518060.00,600000.00,1.Sistema Financeiro de Habitação,300000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16900918084,1.Compra e venda,211000.00,2018-12-21 00:00:00,208709.00,100.00,208709.00,211000.00,1.Sistema Financeiro de Habitação,155000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13611100195,1.Compra e venda,180000.00,2018-12-20 00:00:00,426634.00,100.00,426634.00,426634.00,1.Sistema Financeiro de Habitação,80000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4111100708,1.Compra e venda,910000.00,2018-12-21 00:00:00,547964.00,100.00,547964.00,910000.00,1.Sistema Financeiro de Habitação,250000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4706402441,1.Compra e venda,857142.86,2018-12-20 00:00:00,909231.00,100.00,909231.00,909231.00,1.Sistema Financeiro de Habitação,400000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947162 entries, 0 to 947161
Data columns (total 64 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   N° do Cadastro (SQL)                              913982 non-null  object 
 1   Natureza de Transação                             913982 non-null  object 
 2   Valor de Transação (declarado pelo contribuinte)  913982 non-null  object 
 3   Data de Transação                                 913982 non-null  object 
 4   Valor Venal de Referência                         913982 non-null  float64
 5   Proporção Transmitida (%)                         913982 non-null  float64
 6   Valor Venal de Referência (proporcional)          913982 non-null  float64
 7   Base de Cálculo adotada                           913608 non-null  float64
 8   Tipo de Financiamento                             351905 non-null  object 
 9   Valo

None

In [ ]:
data_types = {
    "N° do Cadastro (SQL)": "string",
    "Natureza de Transação": "string",
    "Valor de Transação (declarado pelo contribuinte)": "float64",
    "Valor Venal de Referência": "float64",
    "Proporção Transmitida (%)": "float64",
    "Valor Venal de Referência (proporcional)": "float64",
    "Base de Cálculo adotada": "float64",
    "Valor Financiado": "float64",
    "Situação do SQL": "string"
}

for col, dtype in data_types.items():
    while True:
        try:
            if dtype == 'string':
                df_combined_itbi[col] = df_combined_itbi[col].astype(str)
            elif dtype == 'Int64':
                df_combined_itbi[col] = pd.to_numeric(df_combined_itbi[col], errors='raise').astype('Int64')
            else:
                df_combined_itbi[col] = df_combined_itbi[col].astype(dtype)
            print(f"Successfully converted {col} to {dtype}")
            break
        except Exception as e:
            print(f"Error converting '{col}' to {dtype}: {e}")
            print(f"Dropping rows in column '{col}' that caused errors and retrying...")
            # Remove as linhas que não podem ser convertidas
            if dtype == 'string':
                mask = pd.isnull(df_combined_itbi[col])
            elif dtype == 'Int64' or dtype == 'float64':
                mask = pd.to_numeric(df_combined_itbi[col], errors='coerce').isnull()
            else:
                mask = pd.isnull(df_combined_itbi[col])
            df_combined_itbi = df_combined_itbi[~mask]
            df_combined_itbi = df_combined_itbi.reset_index(drop=True)
            # Continua tentando novamente após remover linhas inválidas

#display(df_combined_itbi.info())
#display(df_combined_itbi.head())

Successfully converted N° do Cadastro (SQL) to string
Successfully converted Natureza de Transação to string
Error converting 'Valor de Transação (declarado pelo contribuinte)' to float64: could not convert string to float: 'HELENO LOURENCO DA SILVA'
Dropping rows in column 'Valor de Transação (declarado pelo contribuinte)' that caused errors and retrying...
Successfully converted Valor de Transação (declarado pelo contribuinte) to float64
Successfully converted Valor Venal de Referência to float64
Successfully converted Proporção Transmitida (%) to float64
Successfully converted Valor Venal de Referência (proporcional) to float64
Successfully converted Base de Cálculo adotada to float64
Successfully converted Valor Financiado to float64
Successfully converted Situação do SQL to string


In [26]:
df_combined_itbi['N° do Cadastro (SQL)'] = df_combined_itbi['N° do Cadastro (SQL)'].astype(str).str.replace('-', '')
df_combined_itbi['N° do Cadastro (SQL)'] = df_combined_itbi['N° do Cadastro (SQL)'].astype(str).str.zfill(11)

df_combined_itbi['Data de Transação'] = pd.to_datetime(df_combined_itbi['Data de Transação'], errors='coerce')
df_combined_itbi = df_combined_itbi[df_combined_itbi['Data de Transação'].dt.year.between(2019, 2024)]
df_combined_itbi['Ano da Transacao'] = df_combined_itbi['Data de Transação'].dt.year
df_combined_itbi['Mes da Transacao'] = df_combined_itbi['Data de Transação'].dt.month
df_combined_itbi['Ano/Mes da Transacao'] = df_combined_itbi['Data de Transação'].dt.year.astype(str)+'/'+df_combined_itbi['Data de Transação'].dt.month.astype(str).str.zfill(2)
df_combined_itbi['setor'] = df_combined_itbi['N° do Cadastro (SQL)'].astype(str).str[:3]
df_combined_itbi['quadra'] = df_combined_itbi['N° do Cadastro (SQL)'].astype(str).str[3:6]
df_combined_itbi['lote'] = df_combined_itbi['N° do Cadastro (SQL)'].astype(str).str[6:10]
display(df_combined_itbi.head(3))

,N° do Cadastro (SQL),Natureza de Transação,Valor de Transação (declarado pelo contribuinte),Data de Transação,Valor Venal de Referência,Proporção Transmitida (%),Valor Venal de Referência (proporcional),Base de Cálculo adotada,Tipo de Financiamento,Valor Financiado,...,TERRENO,0.3,TERRENO.1,Unnamed: 27,Ano da Transacao,Mes da Transacao,Ano/Mes da Transacao,setor,quadra,lote
127,12310100659,1.Compra e venda,900000.00,2019-01-02,773836.00,100.00,773836.00,900000.00,1.Sistema Financeiro de Habitação,400000.00,...,NaN,NaN,NaN,NaN,2019,1,2019/01,123,101,0065
133,05528502403,1.Compra e venda,420000.00,2019-01-02,303832.00,100.00,303832.00,420000.00,1.Sistema Financeiro de Habitação,340721.07,...,NaN,NaN,NaN,NaN,2019,1,2019/01,055,285,0240
147,29903601032,1.Compra e venda,650000.00,2019-01-03,459364.00,100.00,459364.00,650000.00,NaN,0.00,...,NaN,NaN,NaN,NaN,2019,1,2019/01,299,036,0103


In [35]:
selected_columns = ["N° do Cadastro (SQL)",
                    "Natureza de Transação",
                    "Valor de Transação (declarado pelo contribuinte)",
                    "Data de Transação",
                    "Valor Venal de Referência",
                    "Proporção Transmitida (%)",
                    "Valor Venal de Referência (proporcional)",
                    "Base de Cálculo adotada",
                    "Tipo de Financiamento",
                    "Valor Financiado",
                    "Situação do SQL",
                    "Área do Terreno (m2)",
                    "Área Construída (m2)",
                    "Ano da Transacao",
                    "Mes da Transacao",
                    "Ano/Mes da Transacao",
                    "setor",
                    "quadra",
                    "lote",
                    "bairro"
                    ]
df_combined_itbi = df_combined_itbi[selected_columns]

In [36]:
df_combined_itbi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908503 entries, 0 to 908502
Data columns (total 20 columns):
 #   Column                                            Non-Null Count   Dtype         
---  ------                                            --------------   -----         
 0   N° do Cadastro (SQL)                              908503 non-null  object        
 1   Natureza de Transação                             908503 non-null  object        
 2   Valor de Transação (declarado pelo contribuinte)  908503 non-null  float64       
 3   Data de Transação                                 908503 non-null  datetime64[ns]
 4   Valor Venal de Referência                         908503 non-null  float64       
 5   Proporção Transmitida (%)                         908503 non-null  float64       
 6   Valor Venal de Referência (proporcional)          908503 non-null  float64       
 7   Base de Cálculo adotada                           908331 non-null  float64       
 8   Tipo de Financ

In [38]:
df_combined_itbi = (
    df_combined_itbi
    .merge(
        base_bairro,
        on = 'setor',
        how = 'left'
        )
)

In [39]:
missing_values = df_combined_itbi.isnull().sum()
display(missing_values)

N° do Cadastro (SQL)                                     0
Natureza de Transação                                    0
Valor de Transação (declarado pelo contribuinte)         0
Data de Transação                                        0
Valor Venal de Referência                                0
Proporção Transmitida (%)                                0
Valor Venal de Referência (proporcional)                 0
Base de Cálculo adotada                                172
Tipo de Financiamento                               557102
Valor Financiado                                         0
Situação do SQL                                          0
Área do Terreno (m2)                                     0
Área Construída (m2)                                     0
Ano da Transacao                                         0
Mes da Transacao                                         0
Ano/Mes da Transacao                                     0
setor                                                   

In [40]:
numerical_columns = df_combined_itbi.select_dtypes(include=['int64', 'float64'])
descriptive_stats = numerical_columns.describe()
display(descriptive_stats)

,Valor de Transação (declarado pelo contribuinte),Valor Venal de Referência,Proporção Transmitida (%),Valor Venal de Referência (proporcional),Base de Cálculo adotada,Valor Financiado,Área Construída (m2)
count,908503.00,908503.00,908503.00,908503.00,908331.00,908503.00,908503.00
mean,624546.22,4454465.95,68.98,413847.90,672730.87,104144.32,740.23
std,2828282.89,20350340.46,44.53,2144920.41,2933203.03,183714.54,4487.39
min,0.00,0.00,0.00,0.00,0.00,-604339.00,0.00
25%,215000.00,230383.00,1.43,38985.82,230000.00,0.00,56.00
50%,305489.29,447013.00,100.00,208365.00,330000.00,0.00,105.00
75%,550000.00,1659357.00,100.00,417948.00,600000.00,188000.00,197.00
max,729435080.10,1652865322.00,823.64,488407868.00,729435080.10,18050000.00,225002.00


In [41]:
categorical_columns = df_combined_itbi.select_dtypes(include='object').columns.tolist()
display(categorical_columns)
categorical_columns.remove('N° do Cadastro (SQL)')

for col in categorical_columns:
    print(f"\nValue counts for column: {col}")
    if df_combined_itbi[col].nunique() > 20:
        display(df_combined_itbi[col].value_counts().head(20))
    else:
        display(df_combined_itbi[col].value_counts())

['N° do Cadastro (SQL)',
 'Natureza de Transação',
 'Tipo de Financiamento',
 'Situação do SQL',
 'Área do Terreno (m2)',
 'Ano/Mes da Transacao',
 'setor',
 'quadra',
 'lote',
 'bairro_x',
 'bairro_y']


Value counts for column: Natureza de Transação


Natureza de Transação
1.Compra e venda                                                          821021
12.Dação em pagamento por escritura pública                                20283
20.Realização ou integralização de capital                                 11184
2.Cessão de direitos relativos a compromisso de compra e venda              8959
15.Permuta por escritura pública                                            8568
4.Arrematação (em leilão ou hasta pública)                                  8151
17.Resolução da alienação fiduciária por inadimplemento                     7376
33.Demais atos onerosos translativos                                        6100
3.Adjudicação (vinculada a processo de execução ou cobrança de dívida)      3178
26.Cessão de direitos sobre o imóvel com alienação fiduciária               2617
8.Cessão de direitos hereditários (ou sucessórios)                          1908
5.Adjudicação compulsória                                                   1728
27.Exc


Value counts for column: Tipo de Financiamento


Tipo de Financiamento
1.Sistema Financeiro de Habitação    304268
2.Minha Casa Minha Vida               41666
3.Consórcio                            4405
99.SFI, Carteira Hipotecária, etc      1051
5.                                        8
4.                                        3
Name: count, dtype: int64

...

lote
0001    13683
0002     9948
0004     8808
0006     8715
0003     7000
0009     6969
0007     6604
0015     6547
0005     6155
0011     6134
0019     5966
0008     5773
0012     5758
0021     5336
0020     5271
0032     5239
0016     5197
0010     5109
0017     5070
0024     5048
Name: count, dtype: int64


Value counts for column: bairro_x


bairro_x
nan                       871366
PERDIZES                    8910
ITAQUERA                    4193
CJ HAB RAPOSO TAVARES       2922
CONJ PE M DA NOBREGA        2720
AMERICANOPOLIS              2129
DESDOBRO QD 1 E CD 1 9      1790
CJ HAB JOSE BONIFACIO       1534
ALTO DE PINHEIROS           1194
COHAB ADVENTISTA            1082
FAZENDA DA JUTA             1078
CAPELA DO SOCORRO            814
C H PRESID JUSCELINO         801
STA ETELVINA VII             776
STA ETELVINA IIA             699
TORRE C                      588
A E CARVALHO                 587
FREG DO O                    564
CJ HAB BRIG F LIMA           503
JD BARRO BRANCO I            387
Name: count, dtype: int64


Value counts for column: bairro_y


bairro_y
nan                       871366
PERDIZES                    8910
ITAQUERA                    4193
CJ HAB RAPOSO TAVARES       2922
CONJ PE M DA NOBREGA        2720
AMERICANOPOLIS              2129
DESDOBRO QD 1 E CD 1 9      1790
CJ HAB JOSE BONIFACIO       1534
ALTO DE PINHEIROS           1194
COHAB ADVENTISTA            1082
FAZENDA DA JUTA             1078
CAPELA DO SOCORRO            814
C H PRESID JUSCELINO         801
STA ETELVINA VII             776
STA ETELVINA IIA             699
TORRE C                      588
A E CARVALHO                 587
FREG DO O                    564
CJ HAB BRIG F LIMA           503
JD BARRO BRANCO I            387
Name: count, dtype: int64

In [42]:
transactions_by_month_year = df_combined_itbi.groupby('Ano/Mes da Transacao').size().reset_index(name='Numero de Transacoes')
display(transactions_by_month_year)

,Ano/Mes da Transacao,Numero de Transacoes
0,2019/01,5669
1,2019/02,7129
2,2019/03,7414
3,2019/04,8579
4,2019/05,9203
...,...,...
67,2024/08,20309
68,2024/09,15057
69,2024/10,3998
70,2024/11,17767


In [43]:
df_combined_itbi.to_csv('data/ITBI_2019_2024.csv', index=False)